In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es crear la tabla de la Capa de Oro (Gold),
# uniendo los datos limpios de la Capa de Plata con las dimensiones de negocio.
from pyspark.sql.functions import col, lit

# 1. --- Configuración de Tablas ---
silver_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_cleaned"
dimension_table = "workspace.tecnomundo_data_dimensions.category"
gold_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_categorized"
unmatched_products_table = "workspace.tecnomundo_data_reporting.productos_sin_categoria"

print(f"Tabla de Origen (Plata): {silver_table}")
print(f"Tabla de Dimensiones: {dimension_table}")
print(f"Tabla de Destino (Oro): {gold_table}")
print(f"Tabla de Reporte (Productos no encontrados): {unmatched_products_table}")


# 2. --- Lectura de las Tablas de Origen ---
print(f"\nLeyendo datos desde la Capa de Plata: '{silver_table}'...")
df_silver = spark.table(silver_table)

print(f"Leyendo datos desde la tabla de Dimensiones: '{dimension_table}'...")
df_dimensions = spark.table(dimension_table)


# 3. --- Enriquecimiento de Datos (Categorización) ---
# Ahora que las claves están estandarizadas en las capas anteriores,
# podemos realizar un único y eficiente LEFT JOIN.
print("\nEnriqueciendo datos con la tabla de categorías...")

df_joined = df_silver.join(
    df_dimensions,
    df_silver.codigo_producto == df_dimensions.codigo_producto, # Condición de unión estandarizada
    "left" # Usamos un LEFT JOIN para conservar todas las ventas
)

# 4. --- Separación de Filas Enriquecidas y No Encontradas ---
# Identificamos los productos que no encontraron una categoría en la tabla de verdad.
df_unmatched = df_joined.filter(col("categoria").isNull()).select(df_silver["*"])
df_matched = df_joined.filter(col("categoria").isNotNull())

print(f"Se encontraron {df_matched.count()} filas que fueron categorizadas exitosamente.")
print(f"Se encontraron {df_unmatched.count()} filas con productos no encontrados en la tabla de verdad.")


# 5. --- Preparación de la Tabla de Oro ---
# Seleccionamos y renombramos las columnas para la tabla final.
df_gold = df_matched.select(
    "fecha",
    col("comprobante_num").alias("numero_comprobante"),
    df_silver.codigo_producto, # Usamos la clave de la tabla de plata
    "nombre_del_producto",
    "categoria",
    "cantidad",
    col("precio_un_").alias("precio_unitario"),
    "ganancia",
    "subtotal"
)
print("\nTabla de Oro preparada y lista para guardar.")


# 6. --- Guardado de las Tablas Finales ---
# a) Guardar la tabla de Oro
print(f"\nGuardando la tabla final enriquecida en la Capa de Oro: '{gold_table}'...")
(df_gold.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(gold_table))
print("¡Tabla de la Capa de Oro guardada exitosamente!")

# b) Guardar la tabla de reporte con los productos no encontrados
if df_unmatched.count() > 0:
    print(f"Guardando {df_unmatched.count()} productos no encontrados en la tabla de reporte: '{unmatched_products_table}'...")
    (df_unmatched.select("codigo_producto")
     .distinct() # Guardamos solo una vez cada código de producto no encontrado
     .write
     .mode("overwrite")
     .saveAsTable(unmatched_products_table))
    print("¡Tabla de reporte de productos no encontrados guardada exitosamente!")
else:
    print("¡Excelente! Todos los productos fueron categorizados.")


# 7. --- Verificación Final ---
print("\nMostrando una muestra de la tabla final (Capa de Oro):")
display(spark.table(gold_table))

if df_unmatched.count() > 0:
    print("\nMostrando una muestra de los productos no encontrados:")
    display(spark.table(unmatched_products_table))
